# **SIRD Optimization: The Brazilian Case**

>In order to fit the SIRD model to the reality, we need real data. 
>To do so, accurate data for Brazil could be taken from Wesley Cota's Database, a Postdoctoral Fellow at Universidade Federal de Viçosa - Brazil. \\
* [Wesley Cota](https://github.com/wcota/covid19br)

## Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import optimize
import plotly.graph_objects as go
import yaml
from datetime import datetime
from sklearn import metrics as mt

## Loading

In [3]:
import pandas as pd

#Data importation then conditioning from the Database
data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
data_table = pd.read_csv(data_path, delimiter=",")

data_table.head()

,epi_week,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,recovered,suspects,tests,tests_per_100k_inhabitants,vaccinated,vaccinated_per_100_inhabitants,vaccinated_second,vaccinated_second_per_100_inhabitants,vaccinated_single,vaccinated_single_per_100_inhabitants
0,9,2020-02-25,Brazil,SP,TOTAL,0,0,1,1,0,0,0.0,0.00218,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,2020-02-25,Brazil,TOTAL,TOTAL,0,0,1,1,0,0,0.0,0.00047,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,2020-02-26,Brazil,SP,TOTAL,0,0,0,1,0,1,0.0,0.00218,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,2020-02-26,Brazil,TOTAL,TOTAL,0,0,0,1,0,1,0.0,0.00047,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,2020-02-27,Brazil,SP,TOTAL,0,0,0,1,0,1,0.0,0.00218,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIRD model

In [4]:
# The SIRD model differential equations.
def SIRD(y, t, N, Beta, Gamma, Mu):
    S, I, R, D = y
    dSdt = -(Beta * I * S)/N
    dIdt = (Beta * I * S)/N  - Gamma * I - Mu * I
    dRdt = Gamma * I
    dDdt = Mu * I
    return dSdt, dIdt, dRdt, dDdt

In [5]:
#Saving the simulation results

def SIRDsim(y0, t, N, theta):
  
  #Transmission rate
  Beta = theta[0]
  #Recovery rate per day
  Gamma = theta[1]
  #Mortality rate
  Mu = theta[2]

  # Integrate the SIRD equations over the time grid t.
  result = odeint(SIRD, y0, t, args=(N, Beta, Gamma, Mu))
  S, I, R, D = result.T
  return S, I, R, D

In [6]:
#Least Squares Method

def QuadraticError(theta0, Sd, Id, Rd, Dd, y0, t, N):
    """ function to pass to optimize.leastsq
        The routine will square and sum the values returned by 
        this function""" 
    [S,I,R,D] = SIRDsim(y0, t, N, theta0)
    errorS = S - Sd
    errorI = I - Id
    errorR = R - Rd
    errorD = D - Dd
    EQ = np.concatenate([errorI,errorR,errorD])
    return EQ

## The Case of Brazil

>Even if we have extracted the dataset, we need to filter some information due to the wide scope of the database.

### Raw Data Visualization

In [ ]:
#Data importation then conditioning from the Database
data_file = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'

data_table = pd.read_csv(data_file, delimiter=",")

selected_state = "TOTAL"

#Creating a boolean vector to identify which row is concerning selected_state
#is_state = data_table['state']==selected_state

#Only extracting the useful concerned data
#state_data = data_table[is_state]

#Removing days with lack of information
#state_data = state_data[state_data.recovered.notnull()].reset_index()

#Alternative way: 
state_data = data_table[data_table.state == selected_state].reset_index()

state_data.head()

,index,epi_week,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,recovered,suspects,tests,tests_per_100k_inhabitants,vaccinated,vaccinated_per_100_inhabitants,vaccinated_second,vaccinated_second_per_100_inhabitants,vaccinated_single,vaccinated_single_per_100_inhabitants
0,1,9,2020-02-25,Brazil,TOTAL,TOTAL,0,0,1,1,0,0,0.0,0.00047,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,9,2020-02-26,Brazil,TOTAL,TOTAL,0,0,0,1,0,1,0.0,0.00047,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,9,2020-02-27,Brazil,TOTAL,TOTAL,0,0,0,1,0,1,0.0,0.00047,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,9,2020-02-28,Brazil,TOTAL,TOTAL,0,0,1,2,0,1,0.0,0.00094,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,9,2020-02-29,Brazil,TOTAL,TOTAL,0,0,0,2,0,2,0.0,0.00094,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
fig = go.Figure()

#Recovered
fig.add_trace(go.Scatter(
    name="Recovered",
    x=state_data.date,
    y=state_data.recovered,
    mode='markers',
    marker=dict(size=6, color="limegreen")
))

#Active Cases
active_cases = state_data.totalCases - state_data.recovered
fig.add_trace(go.Scatter(
    name="Active Cases",
    x=state_data.date,
    y=active_cases,
    mode='markers',
    marker=dict(size=6, color="orchid")
))

#Deceased
fig.add_trace(go.Scatter(
    name="Deceased",
    x=state_data.date,
    y=state_data.deaths,
    mode='markers',
    marker=dict(size=6, color="burlywood")
))

fig.update_layout(
    template='xgridoff',
    xaxis=dict(showgrid=False),
    xaxis_title='Date',
    yaxis_title='Individuals',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
    ),
    title_text="Real Data"
)

fig.show()

>> Considering the different curves based on real data, we can deduce that it will be complicated to fit perfectly the SIRD model, since the plot does not seem like a SIRD model simulation. \\
This behaviour can be explained by the fact that reality is clearly more complicated than a simplified SIRD model.

### SIRD fitting

In [ ]:
#Initial Conditions

import yaml
with open('/content/drive/MyDrive/ISMIN/Semestre 8/Stage International/Instituto Maua de Tecnologia/Maua Internship/05_Notebooks/DataSets/state_pop.yaml') as f:
  state_pop = yaml.load(f)

#State population
#N = state_pop['MA']['population']
N = 15e6

#Introduction of the State's Data into the model
Rd = state_data.recovered
Dd = state_data.deaths
Id = state_data.totalCases - Rd
Sd = N - (Id + Rd + Dd)

#Initial values
I0 = 1
R0 = 0
D0 = 0
S0 = N - (I0 + R0 + D0)

#Initial vector
y0 = S0, I0, R0, D0

#Initial parameters
Beta = 0.3
Gamma = 0.15
Mu = 0.0292
theta = [Beta, Gamma, Mu]

#Timeline
t = np.linspace(0, len(state_data), len(state_data))

In [ ]:
# Simulating to find the parameters
(estimated_theta, kvg) = optimize.leastsq(QuadraticError, theta, args=(Sd,Id,Rd,Dd,y0,t,N))
print('The best parameters for this dataset are: ', estimated_theta)

The best parameters for this dataset are:  [0.3    0.15   0.0292]


### Visualization

In [ ]:
#Simulation with best parameters
tsim = np.linspace(0, len(state_data), len(state_data))
[Ss,Is,Rs,Ds] = SIRDsim(y0, tsim, N, estimated_theta) # Finding S, I, R and D for the model

In [ ]:
# Plotting the results
fig = go.Figure()

fig.add_trace(go.Scatter(
    name="Susceptible - Model",
    x=state_data.date,
    y=Ss,
    mode='lines',
    line=dict(width=3, dash="dash", color="#58ACFA")
    ))

fig.add_trace(go.Scatter(
    name="Infected - Model",
    x=state_data,
    y=Is,
    mode='lines',
    line=dict(width=3, dash="dash", color="#6a1b9a")
    ))

fig.add_trace(go.Scatter(
    name="Recovered - Model",
    x=state_data.date,
    y=Rs,
    mode='lines',
    line=dict(width=3, dash="dash", color="#2e7d32")
    ))

fig.add_trace(go.Scatter(
    name="Desceased - Model",
    x=state_data.date,
    y=Ds,
    mode='lines',
    line=dict(width=3, dash="dash", color="burlywood")
    ))

fig.add_trace(go.Scatter(
    name="Susceptible - Data",
    x=state_data.date,
    y=Sd,
    mode='markers',
    marker=dict(size=6, color="#0080FF")
    ))

fig.add_trace(go.Scatter(
    name="Infected - Data",
    x=state_data.date,
    y=Id,
    mode='markers',
    marker=dict(size=6, color="#38006b")
    ))

fig.add_trace(go.Scatter(
    name="Recovered - Data",
    x=state_data.date,
    y=Rd,
    mode='markers',
    marker=dict(size=6, color="#005005")
    ))

fig.add_trace(go.Scatter(
    name="Desceased - Data",
    x=state_data.date,
    y=Dd,
    mode='markers',
    marker=dict(size=6, color="burlywood")
    ))

fig.update_layout(
    template='xgridoff',
    xaxis=dict(showgrid=False),
    xaxis_title='Days',
    yaxis_title='Individuals',
    legend=dict(
      orientation='h',
      yanchor='bottom',
      y=1.01,
      xanchor='right',
      x=0.95
      ),
    title_text="SIR Model")

fig.show()

>> The created method is not optimized. \\
Since the reality is more complex than the model, and in order to get relevant simulations, we need to lead the study only regarding a segment of data. Indeed, if we only study a little part of data, it is simpler to fit the curve. \\
Moreover, we can observe that actual epidemics are not completely free to spread as in the SIRD model. Indeed, if we use the real amount of population to fit the model, it is clear that results are irrelevant, with the *Susceptible* curve staying almost contant at a high position, while other curves are flattened below.
**$$ \color{white}{space} $$**
Finally, building a stronger model is necessary, with the possibility to work on shorter periods, and unfortunately to use a lower amount of population to get workable results.

## Maranhão Example

>In order to design a stronger model, restraining the study to a restricted scale may be helpful. \\
Thus, we will work on the Maranhão example, at the State scale.

In [ ]:
# Reading data - wcota
data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'

df = pd.read_csv(data_path, delimiter=",") 
df.head()

# Dataframe for Maranhão state
state = 'MA'
df_state = df[df.state == state].reset_index()
df_state.head()

,index,epi_week,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,recovered,suspects,tests,tests_per_100k_inhabitants,vaccinated,vaccinated_per_100_inhabitants,vaccinated_second,vaccinated_second_per_100_inhabitants,vaccinated_single,vaccinated_single_per_100_inhabitants
0,220,12,2020-03-20,Brazil,MA,TOTAL,0,0,1,1,0,0,0.0,0.01413,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,247,12,2020-03-21,Brazil,MA,TOTAL,0,0,1,2,0,1,0.0,0.02827,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,275,13,2020-03-22,Brazil,MA,TOTAL,0,0,0,2,0,2,0.0,0.02827,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,303,13,2020-03-23,Brazil,MA,TOTAL,0,0,6,8,0,2,0.0,0.11307,0.0,NaN,302.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,331,13,2020-03-24,Brazil,MA,TOTAL,0,0,0,8,0,8,0.0,0.11307,0.0,NaN,302.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Creating new recovered column
df_state["newRecovered"] = df_state["recovered"].diff()
df_state.newRecovered.fillna(0, inplace=True)
df_state.recovered.fillna(0, inplace=True)

In [ ]:
# Creating active cases column (Infected)
active_infected = [df_state["totalCases"].iloc[0]]
for nc, nr in zip(df_state["newCases"].iloc[1:], 
                  df_state["newRecovered"].iloc[1:]):
    active_infected.append(active_infected[-1] + nc - nr)
df_state["activeCases"] = active_infected

In [ ]:
# Plotting the correct results components
fig = go.Figure()

fig.add_trace(go.Scatter(
    name="Active Cases",
    x=df_state.date,
    y=df_state.activeCases,
    mode='markers',
    marker=dict(size=6, color="#38006b")
    ))

fig.add_trace(go.Scatter(
    name="Recovered",
    x=df_state.date,
    y=df_state.recovered,
    mode='markers',
    marker=dict(size=6, color="#005005")
    ))

fig.add_trace(go.Scatter(
    name="Desceased",
    x=df_state.date,
    y=df_state.deaths,
    mode='markers',
    marker=dict(size=6, color="burlywood")
    ))

fig.update_layout(
    template='xgridoff',
    xaxis=dict(showgrid=False),
    xaxis_title='Date',
    yaxis_title='Individuals',
    legend=dict(
      orientation='h',
      yanchor='bottom',
      y=1.01,
      xanchor='right',
      x=0.95
      ),
    title_text="Real data")

fig.show()

In [ ]:
# States population
with open('/content/drive/MyDrive/ISMIN/Semestre 8/Stage International/Instituto Maua de Tecnologia/Maua Internship/05_Notebooks/DataSets/state_pop.yaml') as f:
    state_pop = yaml.load(f)

In [ ]:
df_state_init = df_state.iloc[:100] # Days to find Parameters 

# Size of population
N = 100000 
#N = state_pop['MA']['population']

# Data
Id = df_state_init["activeCases"]
Rd = df_state_init["recovered"]
Dd = df_state_init["deaths"]
Sd = N - Rd - Id - Dd

In [ ]:
# Initial Values
I0 = 1 
R0 = 0
D0 = 0
S0 = N - I0

# Initial Vector
y0 = S0, I0, R0, D0

# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

# Timeline
t = np.linspace(0, len(df_state_init.index.values), len(df_state_init.index.values))

In [ ]:
# Simulating to find the best parameters
(c, kvg) = optimize.leastsq(QuadraticError, theta0, args=(Sd,Id,Rd,Dd,y0,t,N)) 
print(c)

[0.21582997 0.06459561 0.00648206]


In [ ]:
df_state_sim = df_state.iloc[:250] # Days to simulate
tsim = np.linspace(0, len(df_state_sim.index.values), len(df_state_sim.index.values))
[Ss,Is,Rs,Ds] = SIRDsim(y0,tsim,N,c) # Finding S, I and R for the model

In [ ]:
# Plotting the results
fig = go.Figure()

fig.add_trace(go.Scatter(
    name="Active Cases - Model",
    x=df_state_sim.date,
    y=Is,
    mode='lines',
    line=dict(width=3, dash="dash", color="#6a1b9a")
    ))

fig.add_trace(go.Scatter(
    name="Recovered - Model",
    x=df_state_sim.date,
    y=Rs,
    mode='lines',
    line=dict(width=3, dash="dash", color="#2e7d32")
    ))

fig.add_trace(go.Scatter(
    name="Susceptible - Model",
    x=df_state_sim.date,
    y=Ss,
    mode='lines',
    line=dict(width=3, dash="dash", color="#58ACFA")
    ))

fig.add_trace(go.Scatter(
    name="Desceased - Model",
    x=df_state_sim.date,
    y=Ds,
    mode='lines',
    line=dict(width=3, dash="dash", color="burlywood")
    ))

fig.add_trace(go.Scatter(
    name="Active Cases - Data",
    x=df_state_sim.date,
    y=df_state_sim.activeCases,
    mode='markers',
    marker=dict(size=6, color="#38006b")
    ))

fig.add_trace(go.Scatter(
    name="Recovered - Data",
    x=df_state_sim.date,
    y=Rd,
    mode='markers',
    marker=dict(size=6, color="#005005")
    ))

fig.add_trace(go.Scatter(
    name="Susceptible - Data",
    x=df_state_sim.date,
    y=Sd,
    mode='markers',
    marker=dict(size=6, color="#0080FF")
    ))

fig.add_trace(go.Scatter(
    name="Desceased - Data",
    x=df_state_sim.date,
    y=Dd,
    mode='markers',
    marker=dict(size=6, color="burlywood")
    ))

fig.update_layout(
    template='xgridoff',
    xaxis=dict(showgrid=False),
    xaxis_title='Days',
    yaxis_title='Individuals',
    legend=dict(
      orientation='h',
      yanchor='bottom',
      y=1.01,
      xanchor='right',
      x=0.95
      ),
    title_text="SIR Model")

fig.show()

>> From now on, our method results in relevant data. \\
Indeed, restraining the study to a shorter period of time, on a more restricted area, and using a lower amount of people, simplify the research. \\
However, in order to create an entire database for Brazilian States, new challenges need to be faced. 

### Fitting Evaluation

>The previous figure let us consider that the model is well fitted visually. \\
However, quantifying this fitting precision could be interesting in a continuous way of improvement.

In [ ]:
# Redefining vectors length
df_state = df_state.iloc[:250]
Id = df_state["activeCases"]
Rd = df_state["recovered"]
Dd = df_state["deaths"]
Sd = N - Rd - Id - Dd

# Using R^2 to define precision
mt.r2_score([Sd,Id,Rd,Dd], [Ss,Is,Rs,Ds])

0.7962832955531046

>The R^2 score is a well known tool to quantify the consistancy of two datasets. \\
The objective is to be get a R^2 score the closest to 1. \\
This indicator method could be helpful if we want to improve our fitting precision, doing several iterations for example an keeping the one with the best score.

## Fitting Automation

> Because Brazil is composed of many States, and simulations have to be done several times to actualize the database, creating an automatized universal method is a chance to win time and energy.

In [7]:
def SIRD_fitting_algorithm(State_Name, theta0, Tresearch, Tsim):
  """This function is fitting the SIRD model to a Dataset
     and plots the resulting curves.
     Args:
           - State_Name: Abbreviated name of a State  | string
           - N: State population                      | int
           - theta0: Initial parameters               | tuple of float Beta, Gamma, Mu
           - Tresearch: Parameters' research duration | int (days)
           - Tsim: Simulation duration                | int (days) 
  """

  # Reading data - wcota
  data_path = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
  df = pd.read_csv(data_path, delimiter=",") 

  # Dataframe for the specific State
  df_state = df[df.state == State_Name].reset_index()

  # Creating new recovered column
  df_state["newRecovered"] = df_state["recovered"].diff()
  df_state.newRecovered.fillna(0, inplace=True)
  df_state.recovered.fillna(0, inplace=True)

  # Creating active cases column (Infected)
  active_infected = [df_state["totalCases"].iloc[0]]
  for nc, nr in zip(df_state["newCases"].iloc[1:], 
                    df_state["newRecovered"].iloc[1:]):
      active_infected.append(active_infected[-1] + nc - nr)
  df_state["activeCases"] = active_infected

  # State population
  with open('/content/drive/MyDrive/ISMIN/Semestre 8/Stage International/Instituto Maua de Tecnologia/Maua Internship/05_Notebooks/DataSets/state_pop.yaml') as f:
    state_pop = yaml.load(f)
  N = state_pop[State_Name]['population']

  #Susceptible population
  alpha = (df_state["recovered"][Tresearch] + df_state["activeCases"][Tresearch] + df_state["deaths"][Tresearch])/N
  N = alpha*N
  
  # Considering the research duration for the parameters
  df_state_init = df_state.iloc[:Tresearch]
  
  # Data
  Id = df_state_init["activeCases"]
  Rd = df_state_init["recovered"]
  Dd = df_state_init["deaths"]
  Sd = N - Rd - Id - Dd

  # Initial Conditions
  S0, I0, R0, D0 = N-1, 1, 0, 0
  y0 = S0, I0, R0, D0

  # Time vector
  t = np.linspace(0, len(df_state_init.index.values), len(df_state_init.index.values))

  # Model use to find optimal parameters
  (best_theta, kvg) = optimize.leastsq(QuadraticError, theta0, args=(Sd,Id,Rd,Dd,y0,t,N))

  # Fitted simulation
  df_state_sim = df_state.iloc[:Tsim] # Days of simulation
  tsim = np.linspace(0, len(df_state_sim.index.values), len(df_state_sim.index.values))
  [Ss,Is,Rs,Ds] = SIRDsim(y0, tsim, N, best_theta)

  # Plotting the results
  fig = go.Figure()

  fig.add_trace(go.Scatter(
      name="Active Cases - Model",
      x=df_state_sim.date,
      y=Is,
      mode='lines',
      line=dict(width=3, dash="dash", color="#6a1b9a")
      ))

  fig.add_trace(go.Scatter(
      name="Recovered - Model",
      x=df_state_sim.date,
      y=Rs,
      mode='lines',
      line=dict(width=3, dash="dash", color="#2e7d32")
      ))

  fig.add_trace(go.Scatter(
      name="Susceptible - Model",
      x=df_state_sim.date,
      y=Ss,
      mode='lines',
      line=dict(width=3, dash="dash", color="#58ACFA")
      ))

  fig.add_trace(go.Scatter(
      name="Desceased - Model",
      x=df_state_sim.date,
      y=Ds,
      mode='lines',
      line=dict(width=3, dash="dash", color="burlywood")
      ))

  fig.add_trace(go.Scatter(
      name="Active Cases - Data",
      x=df_state_sim.date,
      y=Id,
      mode='markers',
      marker=dict(size=6, color="#38006b")
      ))
  
  fig.add_trace(go.Scatter(
      name="Recovered - Data",
      x=df_state_sim.date,
      y=Rd,
      mode='markers',
      marker=dict(size=6, color="#005005")
      ))

  fig.add_trace(go.Scatter(
      name="Susceptible - Data",
      x=df_state_sim.date,
      y=Sd,
      mode='markers',
      marker=dict(size=6, color="#0080FF")
      ))

  fig.add_trace(go.Scatter(
      name="Desceased - Data",
      x=df_state_sim.date,
      y=Dd,
      mode='markers',
      marker=dict(size=6, color="burlywood")
      ))
  
  fig.update_layout(
      template='xgridoff',
      xaxis=dict(showgrid=False),
      xaxis_title='Days',
      yaxis_title='Individuals',
      legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.01,
        xanchor='right',
        x=0.95
        ),
      title_text="SIR Model")

  fig.show()
  return best_theta

## States

### Acre

In [ ]:
# Initial Parameters
theta0 = [0.4, 0.1, 0.002]

SIRD_fitting_algorithm('AC', theta0, 200, 300)

array([0.13099162, 0.03978684, 0.00244559])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:05 using Wesley Cota Database


### Alagoas

In [ ]:
# Initial Parameters
theta0 = [0.2, 0.09, 0.01]

SIRD_fitting_algorithm('AL', theta0, 170, 300)

array([0.18996589, 0.0914129 , 0.00086584])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:06 using Wesley Cota Database


### Amapá

In [ ]:
# Initial Parameters
theta0 = [0.3, 0.2, 0.002]

SIRD_fitting_algorithm('AP', theta0, 200, 300)

array([0.13690267, 0.02874346, 0.00303998])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:07 using Wesley Cota Database


### Amazonas

In [ ]:
# Initial Parameters
theta0 = [0.2, 0.1, 0.003]

SIRD_fitting_algorithm('AM', theta0, 200, 300)

array([0.26485638, 0.14353721, 0.00625179])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:07 using Wesley Cota Database


### Bahia

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('BA', theta0, 100, 300)

array([0.16067174, 0.0355652 , 0.00127145])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:08 using Wesley Cota Database


### Ceará

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('CE', theta0, 150, 300)

array([0.23839702, 0.107653  , 0.00800654])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:09 using Wesley Cota Database


### Distrito Federal

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('DF', theta0, 140, 300)

array([ 0.1885235 ,  0.09053376, -0.00267606])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:09 using Wesley Cota Database


### Espírito Santo

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('ES', theta0, 130, 300)

array([0.16252771, 0.05624014, 0.00144072])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:10 using Wesley Cota Database


### Goiás

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('GO', theta0, 150, 300)

array([0.1476407 , 0.05438506, 0.00052781])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:10 using Wesley Cota Database


### Maranhão

In [8]:
#Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('MA', theta0, 200, 300)

array([0.21691681, 0.09964236, 0.00286476])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:11 using Wesley Cota Database


### Mato Grosso

In [ ]:
# Initial Parameters
theta0 = [0.1, 0.1, 0.0292]

SIRD_fitting_algorithm('MT', theta0, 120, 200)

/usr/local/lib/python3.7/dist-packages/scipy/integrate/odepack.py:248: ODEintWarning:

Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.



array([ 1.32019182e-01,  3.12016942e-02, -1.97404181e-05])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:12 using Wesley Cota Database


### Mato Grosso do Sul

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('MS', theta0, 130, 200)

array([ 0.14111787,  0.05483433, -0.00130128])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:13 using Wesley Cota Database


### Minas Gerais

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('MG', theta0, 125, 200)

array([ 0.16423401,  0.06328051, -0.00060204])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:13 using Wesley Cota Database


### Pará

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('PA', theta0, 125, 300)

array([0.27109101, 0.12880479, 0.00471953])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:14 using Wesley Cota Database


### Paraíba

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('PB', theta0, 150, 300)

array([0.12920471, 0.02073467, 0.00201256])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:14 using Wesley Cota Database


### Paraná

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('PR', theta0, 125, 300)

array([ 0.1156256 ,  0.01941321, -0.00070422])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:15 using Wesley Cota Database


### Pernambuco

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('PE', theta0, 125, 300)

array([0.19116264, 0.05126888, 0.00896695])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:16 using Wesley Cota Database


### Piauí

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('PI', theta0, 125, 300)

array([ 0.21443541,  0.10823948, -0.00257877])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:16 using Wesley Cota Database


### Rio de Janeiro

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('RJ', theta0, 200, 300)

array([0.21936729, 0.10711373, 0.0087136 ])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:17 using Wesley Cota Database


### Rio Grande do Norte

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('RN', theta0, 125, 300)

array([0.10848669, 0.00350008, 0.0007154 ])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:18 using Wesley Cota Database


### Rio Grande do Sul

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('RS', theta0, 125, 300)

array([ 0.21911646,  0.11522734, -0.00127709])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:19 using Wesley Cota Database


### Rondônia

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('RO', theta0, 125, 300)

array([0.15080173, 0.04013762, 0.00249444])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:19 using Wesley Cota Database


### Roraima

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('RR', theta0, 125, 300)

array([0.12058866, 0.01854736, 0.00045444])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:20 using Wesley Cota Database


### Santa Catarina

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('SC', theta0, 125, 300)

array([ 0.19432884,  0.0918351 , -0.00059081])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:21 using Wesley Cota Database


### São Paulo

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('SP', theta0, 150, 300)

array([0.15076866, 0.03462773, 0.004313  ])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:21 using Wesley Cota Database


### Sergipe

In [ ]:
# Initial Parameters
theta0 = [1e-1, 1e-2, 1e-3]

SIRD_fitting_algorithm('SE', theta0, 200, 300)

array([0.13851427, 0.04586808, 0.00064514])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:22 using Wesley Cota Database


### Tocantins

In [ ]:
# Initial Parameters
theta0 = [0.44, 0.15, 0.0292]

SIRD_fitting_algorithm('TO', theta0, 300, 350)

array([0.11647243, 0.04632247, 0.00206006])

In [ ]:
simulation_time = datetime.now().strftime("%d/%m/%Y at %H:%M:%S")
print("Simulation done the", simulation_time, "using Wesley Cota Database")

Simulation done the 29/06/2021 at 21:04:22 using Wesley Cota Database


## To go further

>Through this Notebook, we have improved our fitting process concerning the SIRD model. As a result, we can now use this method on any bunch of data (any Brazilian State for example), and get workable results. \\
However, to obtain accurate results, a large amount of time need to be spent on playing with the parameters and initial conditions. \\
To improve our process again and save energy and time, 2 tracks can be followed: 
1. Machine Learning utilization, in order to find the best parameters
1. Brut Force utilization, using an evaluation method like $R^2$ to save the best settings